# Cricket Match Innings

In [47]:
import numpy as np
import pandas as pd
import json
import http.client
import time  
# List of match IDs

match_ids = [
    13265827, 13265828, 13265829, 13265830, 13265831, 13265832, 13265833, 
    13265834, 13265836, 13265837, 13265838, 13265840, 13558575, 13558576, 13569002
]

# API connection setup
conn = http.client.HTTPSConnection("allsportsapi2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "693b14936cmshb4229d5afd9d447p186b67jsn499928bbef0a",
    'x-rapidapi-host': "allsportsapi2.p.rapidapi.com"
}

# Parent dictionary to store all match data
parent_json = {}

# Loop through each match ID
for match_id in match_ids:
    try:
        # Make API request
        conn.request("GET", f"/api/cricket/match/{match_id}/innings", headers=headers)
        res = conn.getresponse()
        data = res.read()
        
        # Convert response to JSON
        data_json = json.loads(data.decode("utf-8"))
        
        # Store JSON in the parent dictionary with match_id as key
        parent_json[f"match_innings_{match_id}"] = data_json

        print(f"Data collected for Match ID {match_id}")

    except Exception as e:
        print(f"Error fetching data for Match ID {match_id}: {e}")

    time.sleep(1)  # Optional: Delay to prevent rate limits

# Save all match data in a single JSON file
with open("all_match_innings_data.json", "w", encoding="utf-8") as f:
    json.dump(parent_json, f, indent=4)

print("All match data saved in all_match_innings_data.json")

Data collected for Match ID 13265827
Data collected for Match ID 13265828
Data collected for Match ID 13265829
Data collected for Match ID 13265830
Data collected for Match ID 13265831
Data collected for Match ID 13265832
Error fetching data for Match ID 13265833: Expecting value: line 1 column 1 (char 0)
Data collected for Match ID 13265834
Error fetching data for Match ID 13265836: Expecting value: line 1 column 1 (char 0)
Data collected for Match ID 13265837
Data collected for Match ID 13265838
Data collected for Match ID 13265840
Data collected for Match ID 13558575
Data collected for Match ID 13558576
Data collected for Match ID 13569002
All match data saved in all_match_innings_data.json


## Loading Json file

In [73]:
import numpy as np
import pandas as pd
import json
with open("all_match_innings_data.json", "r", encoding="utf-8") as f:
    all_match_innings = json.load(f)
all_match_innings

{'match_innings_13265827': {'innings': [{'number': 1,
    'battingTeam': {'name': 'New Zealand',
     'slug': 'new-zealand',
     'shortName': 'NZ',
     'gender': 'M',
     'sport': {'name': 'Cricket', 'slug': 'cricket', 'id': 62},
     'userCount': 14667,
     'nameCode': 'NZE',
     'disabled': False,
     'national': True,
     'type': 0,
     'id': 187755,
     'entityType': 'team',
     'teamColors': {'primary': '#ffffff',
      'secondary': '#000000',
      'text': '#000000'},
     'fieldTranslations': {'nameTranslation': {'ar': 'نيوزيلندا',
       'hi': 'न्यूज़ीलैंड',
       'bn': 'নিউজিল্যান্ড'},
      'shortNameTranslation': {'ar': 'نيوزيلندا',
       'hi': 'एनज़ेड',
       'bn': 'এনজেড'}}},
    'bowlingTeam': {'name': 'Pakistan',
     'slug': 'pakistan',
     'shortName': 'PAK',
     'gender': 'M',
     'sport': {'name': 'Cricket', 'slug': 'cricket', 'id': 62},
     'userCount': 20377,
     'nameCode': 'PAK',
     'disabled': False,
     'national': True,
     'type': 0,
   

# Depth json to flatten_json

In [74]:
# Function for Convert a nested json dictionary to flatten json or dictionary
def flatten_json(nested_json, parent_key='', sep='_'):

    items = []
    
    # If the value is a dictionary
    if isinstance(nested_json, dict):
        for key, value in nested_json.items():
            new_key = f"{parent_key}{sep}{key}" if parent_key else key
            items.extend(flatten_json(value, new_key, sep=sep).items())
    
    # If the value is a list
    elif isinstance(nested_json, list): 
        for index, item in enumerate(nested_json):
            new_key = f"{parent_key}{sep}{index}" if parent_key else str(index)
            items.extend(flatten_json(item, new_key, sep=sep).items())
            
    # If the value is neither dict nor list
    else:
        items.append((parent_key, nested_json))

    return dict(items)

# Flatten Json to Row column

In [75]:
def process_json_file(data):
    try:
        # Normalize JSON data
        if isinstance(data, list):  # JSON is an array of objects
            flat_data = [flatten_json(item) for item in data]
        elif isinstance(data, dict):  # JSON is a single dictionary
            flat_data = [flatten_json(data)]
        else:
            raise ValueError("Unsupported JSON format")

        # Convert to DataFrame
        df = pd.DataFrame(flat_data)
        return df


    except json.JSONDecodeError:
        print("Error: Invalid JSON file.")
    except Exception as e:
        print(f"Error: {str(e)}")

## 1. first level keys

In [76]:
# first level keys
print("Data Type: ", type(all_match_innings))
all_match_innings.keys()

Data Type:  <class 'dict'>


dict_keys(['match_innings_13265827', 'match_innings_13265828', 'match_innings_13265829', 'match_innings_13265830', 'match_innings_13265831', 'match_innings_13265832', 'match_innings_13265834', 'match_innings_13265837', 'match_innings_13265838', 'match_innings_13265840', 'match_innings_13558575', 'match_innings_13558576', 'match_innings_13569002'])

# Process my parent dict ot file

In [ ]:
def process_dictionary(data):
    all_dataframe = {}
    parent_json = {}

    for key in data:
        if data[key] is not None and data[key] != "" and data[key] != {}:
            if isinstance(data[key], list):  # If value is a list
                dataframe = pd.DataFrame(columns=process_json_file(flatten_json(data[key][0])).columns)
                for i in range(len(data[key])):
                    flatten_dict = flatten_json(data[key][i])
                    new_dataframe = process_json_file(flatten_dict)
                    dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)

                    # Deleting field translation columns
                    dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 10:  # If columns are 10 or more
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]

            elif isinstance(data[key], dict):  # If value is a dictionary
                flatten_dict = flatten_json(data[key])
                dataframe = process_json_file(flatten_dict)

                # Deleting field translation columns
                dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 5:  # If columns are 5 or more
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]
            else:
                parent_json[key] = data[key]

    # Convert parent_json into a DataFrame and store it in all_dataframe
    all_dataframe["match_stats"] = process_json_file(flatten_json(parent_json))

    return all_dataframe

In [78]:
battingTeam = pd.DataFrame()
bowlingTeam = pd.DataFrame()
battingLine = pd.DataFrame()
bowlingLine = pd.DataFrame()
partnerships = pd.DataFrame()
match_stats = pd.DataFrame()

In [79]:
for i in all_match_innings.keys():
    match_id = int(i.split('_')[-1])
    data = all_match_innings[i]
    
    # extracting innings1 and innings2 data separately from parent data
    innings1_dataframes = process_dictionary(data['innings'][0])
    innings2_dataframes = process_dictionary(data['innings'][1])

    # storing data properly
    # ******************************** For innings 1 ************************************
    # battingTeam
    innings1_battingTeam = innings1_dataframes['battingTeam']
    innings1_battingTeam['match_id'] = match_id
    innings1_battingTeam['innings'] = 1
    battingTeam = pd.concat([battingTeam, innings1_battingTeam], ignore_index=True)

    # bowlingTeam
    innings1_bowlingTeam = innings1_dataframes['bowlingTeam']
    innings1_bowlingTeam['match_id'] = match_id
    innings1_bowlingTeam['innings'] = 1
    bowlingTeam = pd.concat([bowlingTeam, innings1_bowlingTeam], ignore_index=True)

    # battingLine
    innings1_battingLine = innings1_dataframes['battingLine']
    innings1_battingLine['match_id'] = match_id
    innings1_battingLine['innings'] = 1
    battingLine = pd.concat([battingLine, innings1_battingLine], ignore_index=True)

    # bowlingLine
    innings1_bowlingLine = innings1_dataframes['bowlingLine']
    innings1_bowlingLine['match_id'] = match_id
    innings1_bowlingLine['innings'] = 1
    bowlingLine = pd.concat([bowlingLine, innings1_bowlingLine], ignore_index=True)

    # partnerships
    innings1_partnerships = innings1_dataframes['partnerships']
    innings1_partnerships['match_id'] = match_id
    innings1_partnerships['innings'] = 1
    partnerships = pd.concat([partnerships, innings1_partnerships], ignore_index=True)

    # match_stats
    innings1_match_stats = innings1_dataframes['match_stats']
    innings1_match_stats['match_id'] = match_id
    innings1_match_stats['innings'] = 1
    match_stats = pd.concat([match_stats, innings1_match_stats], ignore_index=True)


    # ***************************** For innings 2 *********************************

    # battingTeam
    innings2_battingTeam = innings2_dataframes['battingTeam']
    innings2_battingTeam['match_id'] = match_id
    innings2_battingTeam['innings'] = 2
    battingTeam = pd.concat([battingTeam, innings2_battingTeam], ignore_index=True)

    # bowlingTeam
    innings2_bowlingTeam = innings2_dataframes['bowlingTeam']
    innings2_bowlingTeam['match_id'] = match_id
    innings2_bowlingTeam['innings'] = 2
    bowlingTeam = pd.concat([bowlingTeam, innings2_bowlingTeam], ignore_index=True)

    # battingLine
    innings2_battingLine = innings2_dataframes['battingLine']
    innings2_battingLine['match_id'] = match_id
    innings2_battingLine['innings'] = 2
    battingLine = pd.concat([battingLine, innings2_battingLine], ignore_index=True)


    # bowlingLine
    innings2_bowlingLine = innings2_dataframes['bowlingLine']
    innings2_bowlingLine['match_id'] = match_id
    innings2_bowlingLine['innings'] = 2
    bowlingLine = pd.concat([bowlingLine, innings2_bowlingLine], ignore_index=True)

    # partnerships
    innings2_partnerships = innings2_dataframes['partnerships']
    innings2_partnerships['match_id'] = match_id
    innings2_partnerships['innings'] = 2
    partnerships = pd.concat([partnerships, innings2_partnerships], ignore_index=True)

    # match_stats
    innings2_match_stats = innings2_dataframes['match_stats']
    innings2_match_stats['match_id'] = match_id
    innings2_match_stats['innings'] = 2
    match_stats = pd.concat([match_stats, innings2_match_stats], ignore_index=True)
    

C:\Program Files\KMSpico\temp\ipykernel_8548\3303925257.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)
C:\Program Files\KMSpico\temp\ipykernel_8548\3303925257.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)
C:\Program Files\KMSpico\temp\ipykernel_8548\3303925257.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entri

# Saving all files

In [81]:
battingTeam.to_excel("battingTeam.xlsx", index=False)

In [83]:
bowlingTeam.to_excel("bowlingTeam.xlsx", index=False)

In [85]:
battingLine.to_excel('battingLine.xlsx', index=False)

In [87]:
bowlingLine.to_excel('bowlingLine.xlsx', index=False)

In [89]:
partnerships.to_excel('partnerships.xlsx', index=False)

In [91]:
match_stats.to_excel('match_stats.xlsx', index=False)

# Saving file code

In [15]:
import pandas as pd

output_file = "innings_1.xlsx"  # Define output Excel file

for i, (key, df) in enumerate(all_dataframe.items()):
    print(f"Saving data for {key} in {output_file}")
    
    # If it's the first DataFrame, write with mode 'w' (write mode) to create the file
    # For the rest, use mode 'a' (append) without re-writing the file
    with pd.ExcelWriter(output_file, engine="openpyxl", mode='w' if i == 0 else 'a') as writer:
        df.to_excel(writer, sheet_name=key[:31], index=False)  # Ensure sheet name is within 31 chars

print(f"All DataFrames have been saved in {output_file}")

Saving data for battingTeam in innings_1.xlsx
Saving data for bowlingTeam in innings_1.xlsx
Saving data for bowlingLine in innings_1.xlsx
Saving data for battingLine in innings_1.xlsx
Saving data for partnerships in innings_1.xlsx
Saving data for parent_df in innings_1.xlsx
All DataFrames have been saved in innings_1.xlsx
